In [42]:
import os
from googleapiclient.discovery import build
import pandas as pd
from google.oauth2 import service_account
from IPython.display import JSON, display

In [24]:
credentials = os.environ.get('YOUTUBE_API_KEY')

In [28]:
credentials = os.environ.get('YOUTUBE_API_KEY')
api_service_name = "youtube"
api_version = "v3"


# Create an API client
youtube = build(api_service_name, api_version, developerKey=credentials)

In [32]:
channel_ids = ['UCq6VFHwMzcMXbuKyG7SQYIg']

In [35]:
request = youtube.channels().list(
    part="snippet,contentDetails,statistics",
    id="UCq6VFHwMzcMXbuKyG7SQYIg"
)

response = request.execute()

In [45]:
JSON(response)




<IPython.core.display.JSON object>

In [48]:
def get_channel_statistic(youtube, channel_ids):
    complete_data = []
    request = youtube.channels().list(
    part="snippet,contentDetails,statistics",
    id=",".join(channel_ids)
    )

    response = request.execute()
    for item in response['items']:
        data = {
            'channel_name': item['snippet']["title"],
            'channel_created_at': item['snippet']["publishedAt"],
            'total_viewer': item['statistics']['viewCount'],
            'total_subscriber': item['statistics']['subscriberCount'],
            'total_video': item['statistics']['videoCount'],
            'playlist_id': item['contentDetails']['relatedPlaylists']['uploads']
        }
        complete_data.append(data)

    return(pd.DataFrame(complete_data))

In [49]:
get_channel_statistic(youtube, channel_ids)

,channel_name,channel_created_at,total_viewer,total_subscriber,total_video,playlist_id
0,penguinz0,2007-05-07T23:42:34Z,8335914997,13900000,4792,UUq6VFHwMzcMXbuKyG7SQYIg


In [64]:
def get_video_ids(youtube, playlist_id):
    index=0
    video_id_list = []
    request = youtube.playlistItems().list(
        part='snippet,contentDetails',
        playlistId=playlist_id,
        maxResults = 50
    )
    response = request.execute()
    for item in response['items']:
        video_id_list.append(item['contentDetails']['videoId'])
    next_page_token = response.get('nextPageToken')

    while next_page_token is not None:
        request = youtube.playlistItems().list(
        part='snippet,contentDetails',
        playlistId=playlist_id,
        maxResults = 50,
        pageToken = next_page_token
        )
        response = request.execute()
        for item in response['items']:
            video_id_list.append(item['contentDetails']['videoId'])
        next_page_token = response.get('nextPageToken')
        index+=1
        print('NextToken:{}-{}'.format(index,next_page_token))

    return video_id_list


In [65]:
video_ids = get_video_ids(youtube, 'UUq6VFHwMzcMXbuKyG7SQYIg')

NextToken:1-EAAaIlBUOkNKWUJJaEEwTmpFMk56WTJOa1JDUVRWRE56QkVLQUU
NextToken:2-EAAaIlBUOkNNZ0JJaEEzUmpSR01UWkVRelE1UXpBelJqQTFLQUU
NextToken:3-EAAaIlBUOkNQb0JJaEE0UWtRME1qTkdNVUUxTXpaQ05rWXdLQUU
NextToken:4-EAAaIlBUOkNLd0NJaEJETUVNNE0wWTJNRGs0UkRRMk9ERTJLQUU
NextToken:5-EAAaIlBUOkNONENJaEJDTTBWQlJrUTFSVFkwTjBGR09VVkdLQUU
NextToken:6-EAAaIlBUOkNKQURJaEEyUWpNNU9FVTJSVGhETVRNNU5EQXdLQUU
NextToken:7-EAAaIlBUOkNNSURJaEJCUWpoR09EQTRNelZGUkVJelF6QTBLQUU
NextToken:8-EAAaIlBUOkNQUURJaEEyTjBGRFJVUTROREpGUXpSR1JUTTJLQUU
NextToken:9-EAAaIlBUOkNLWUVJaEEzUXpkQ1JFWXhNVFE1T0RJM1EwWkJLQUU
NextToken:10-EAAaIlBUOkNOZ0VJaEF3UmtJNU9ETkZNRUV4TlRRNE9EYzJLQUU
NextToken:11-EAAaIlBUOkNJb0ZJaEEzT0RFNFJqSXdRalkxUmpsQlEwWkJLQUU
NextToken:12-EAAaIlBUOkNMd0ZJaEF6UkVNMU1qQXdOME5HUTBFek16RkNLQUU
NextToken:13-EAAaIlBUOkNPNEZJaEJETlVOR01FSTJOVE0zUWpjMFF6ZzRLQUU
NextToken:14-EAAaIlBUOkNLQUdJaEEwT0ROQ01EQkdSamcxUWtOR016ZENLQUU
NextToken:15-EAAaIlBUOkNOSUdJaEEwTkVFMVEwRTRNRU13UlRBd1JqVTVLQUU
NextToken:16-EAAaIlBUOkNJUUhJaEF6U

In [67]:
len(video_ids)

4735

In [68]:
# Save List to files
# import pickle
# with open('video_ids.pkl', 'wb') as file:
#     pickle.dump(video_ids, file)

In [83]:
def get_video_detailed_list(youtube, video_ids):
    video_details_list = []

    for i in range(0, len(video_ids), 50):
        request = youtube.videos().list(
            part='snippet,contentDetails,statistics',
            id=','.join(video_ids[i:i+50])
        )
        response = request.execute()

        for video in response['items']:
            stats_to_keep = {
                            'snippet': ['channelTitle', 'title', 'description', 'tags', 'publishedAt'],
                            'statistics': ['viewCount', 'likeCount', 'favouriteCount', 'commentCount'],
                            'contentDetails': ['duration', 'definition', 'caption']
                            }

            video_details = {}
            video_details['id'] = video['id']

            for k in stats_to_keep.keys():
                for v in stats_to_keep[k]:
                    try:
                        video_details[v] = video[k][v]
                    except:
                        video_details[v] = None
            video_details_list.append(video_details)
        print("Videos collected:{}".format(i))
    return pd.DataFrame(video_details_list)

In [84]:
video_df = get_video_detailed_list(youtube, video_ids)

Videos collected:0
Videos collected:50
Videos collected:100
Videos collected:150
Videos collected:200
Videos collected:250
Videos collected:300
Videos collected:350
Videos collected:400
Videos collected:450
Videos collected:500
Videos collected:550
Videos collected:600
Videos collected:650
Videos collected:700
Videos collected:750
Videos collected:800
Videos collected:850
Videos collected:900
Videos collected:950
Videos collected:1000
Videos collected:1050
Videos collected:1100
Videos collected:1150
Videos collected:1200
Videos collected:1250
Videos collected:1300
Videos collected:1350
Videos collected:1400
Videos collected:1450
Videos collected:1500
Videos collected:1550
Videos collected:1600
Videos collected:1650
Videos collected:1700
Videos collected:1750
Videos collected:1800
Videos collected:1850
Videos collected:1900
Videos collected:1950
Videos collected:2000
Videos collected:2050
Videos collected:2100
Videos collected:2150
Videos collected:2200
Videos collected:2250
Videos coll

In [86]:
# Save the video detail dataframe
# video_df
# with open('video_df.pkl', 'wb') as file:
    # pickle.dump(video_df, file)

In [87]:
video_df

,id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,favouriteCount,commentCount,duration,definition,caption
0,J_MamX_v_Zg,penguinz0,Twitch vs Youtube,This is the greatest streaming platform inform...,None,2023-09-29T18:15:01Z,1619313,74450,None,3535,PT17M9S,hd,false
1,KJFObIJk9FY,penguinz0,Evil Wife,This is the greatest video evidence of All Tim...,None,2023-09-29T00:30:09Z,3684374,171816,None,10252,PT10M45S,hd,false
2,ICBFRqZ8zxs,penguinz0,Pumpkin Spice Tier List,Get the Starforge Godslap PC here https://star...,None,2023-09-28T19:30:03Z,1053578,40867,None,2259,PT37M55S,hd,false
3,yPr8p9PD634,penguinz0,iPhone 15 Drama,This is the greatest durability test of All Ti...,None,2023-09-27T19:00:35Z,3411464,145995,None,10844,PT10M2S,hd,false
4,_xmmhV5lO2M,penguinz0,Getting goofy,Merch https://moistglobal.com/\nComics https:/...,None,2023-09-27T05:40:42Z,380474,9407,None,228,PT2H52M41S,hd,false
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4730,tUqqYskyszo,penguinz0,Halo 3- Hayabusa Armor tutorial,get IWHBYD skull:\nyou must jump throught the ...,"[halo, hayabusa, armor, tutorial, MoB, InsAnE]",2007-10-03T01:25:13Z,359548,3321,None,859,PT6M46S,sd,false
4731,lIQpKQ1dIqc,penguinz0,Halo 3- full hayabusa armor,PLEASE READ: I'm tired of gettin messages aski...,"[halo, ful, hayabusa, armor]",2007-09-30T14:26:27Z,248102,2903,None,665,PT31S,sd,false
4732,tQbkowZSEvk,penguinz0,skate- freeskate in the x games,free skating in the x games,"[skate, games, free]",2007-09-19T11:00:44Z,392576,3333,None,526,PT1M26S,sd,false
4733,ad4qga-xNDU,penguinz0,Cr1TiKaL's 1st sniper montage,when i made this video my gamertag was MoB o I...,"[mob, insane, gow, gears, of, war, headshot, m...",2007-08-14T19:39:56Z,534852,15328,None,2150,PT3M10S,sd,false


In [90]:
def get_comments_in_videos(youtube, video_ids):
    # Get 10 top comment on each videos.
    all_comments = []

    for video_id in video_ids:
        try:
            request = youtube.commentThreads().list(
                part="snippet,replies",
                videoId=video_id
            )
            response = request.execute()

            comments = [comment['snippet']['topLevelComment']['snippet']['textOriginal'] for comment in response['items'][0:10]]
            comments_info = {'video_id': video_id, 'comments': comments}

            all_comments.append(comments_info)

        except:
            print('Error during collecting:' + video_id)

    return pd.DataFrame(all_comments)

In [91]:
comments_df = get_comments_in_videos(youtube, video_ids)

Error during collectiingsszs-5bGJQg
Error during collectiingl_w_GHJ5dW4
Error during collectiingrrq-cTAA2Zk
Error during collectiingJBkMB9e4zgI
Error during collectiing8HHqq1wzd3c


In [93]:
# Save the comment dataframe
comments_df
with open('comments.pkl', 'wb') as file:
    pickle.dump(comments_df, file)
